In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from strategy_v2.Strategy import *
from strategy_v2.Strategy.MVO import *
from strategy_v2.Strategy.MVO.AlphaModel import *
from strategy_v2.Strategy.MVO.RiskModel import *
from strategy_v2.TradingSubSystem import *
from strategy_v2.Portfolio import *
from strategy_v2.TransactionCost import *
from strategy_v2.Executor import *
from utils.data_helper import *
from utils.data import *
from utils.performance import *

In [2]:
instruments = [
    'META',
    'TSLA',
    'NVDA',
    'AAPL',
    'EWY',
    'DXJ',
    'BRK-B',
    'SPY',
    'QQQ'
]

end_date = get_today()
start_date = pd.to_datetime(datetime(2022, 1, 3))
max_leverage = 1
vol_target = 0.25

# Notes

- 2024-08-28: Tested expected return prediction with RandomForest, XGB, LGBM. None of them outperforms the SMA Model. LGBM is able to achieve a similar performance as SMA model and generally train faster.</br>

    | Measure                | ^SPX      | MVO - SMA1 | MVO - RandomForest1 | MVO - XGB1 | MVO - LGBM1 |
    |------------------------|-----------|------------|---------------------|------------|-------------|
    | Cumulative Return      | 1.173503  | 2.072994   | 1.662573            | 1.471717   | 2.023378    |
    | Annualized Return      | 0.076775  | 0.301537   | 0.221185            | 0.176047   | 0.296252    |
    | Annualized Volatility  | 0.180081  | 0.226417   | 0.240133            | 0.243953   | 0.243120    |
    | Annualized Sharpe Ratio| 0.240438  | 1.183923   | 0.781685            | 0.584416   | 1.080844    |
    | Maximum Drawdown       | -0.254251 | -0.271732  | -0.323658           | -0.380737  | -0.296535   |

    Models are shared the same hyperparameters: lookback (train days) = 10 days and gamma=10, hhi=0.2


In [8]:
portfolio = PortfolioStandard(
    capital=169336.48958837058,
    name='MVOPortfolio',        
    rebalance_iter=RebalancerIter('0 0 * * Fri', 2),        
    tc_model=TransactionCostFutu(), 
    systems=[        
        #TradingSubSystemSingle(vol_target=vol_target, instruments=instruments,  strategy=[MeanVarianceOpt(alpha_model=LGBM(5,10), risk_model=RollingMeanCov(60), gamma=10, hhi=0.2, lookback=60, confidence=1)], max_leverage=max_leverage, offset=60),
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments,  strategy=[MeanVarianceOpt(alpha_model=RollingMean(10), risk_model=RollingMeanCov(60), gamma=10, hhi=0.2, lookback=60, confidence=1)], max_leverage=max_leverage, offset=60),        
    ]
)

portfolio.set_start_date(start_date)
portfolio.set_end_date(end_date)
portfolio.backtest_subsystems()
portfolio.optimize()
portfolio.backtest()
portfolio.rebalance()
portfolio.performance(show_all_rets=True)

2024-09-02 01:35:29,273 - TradingSubSystemSingle [MVO - RollingMean1] - INFO - Generating position for strategy MVO - RollingMean1 between 2021-12-31 and 2024-09-01......
100%|██████████| 696/696 [00:04<00:00, 172.12it/s]
2024-09-02 01:35:33,337 - TradingSubSystemSingle [MVO - RollingMean1] - INFO - Volatility Target = 25.0% | Price Volatility = 24.8% | Last Scale Factor = 0.98


,Rebalanced Portfolio,Optimized Portfolio,^SPX,MVO - RollingMean1
Measure,,,,
Cumulative Return,1.946831,1.805090,1.177594,1.867697
Annualized Return,0.276611,0.247886,0.077767,0.259596
Annualized Volatility,0.226638,0.225394,0.179817,0.220230
Annualized Sharpe Ratio,1.072786,0.951261,0.246308,1.026743
Maximum Drawdown,-0.325118,-0.306475,-0.254251,-0.289573


In [4]:
portfolio.get_position_for_trade()

2024-08-27 02:12:11,586 - Standard Portfolio ({self.name}) - CRITICAL - Portfolio target capital is not specified, use initial backtest capital of $169,336
2024-08-27 02:12:11,587 - Standard Portfolio ({self.name}) - INFO - Generate trade position based on target capital of $169,336


""


# Execute the Portfolio

In [168]:
# executor = ExecutorFutu(is_test=True)
# executor.set_portfolio(portfolio)
# orders = executor.execute(px_interval='5m')
# orders

# Calibration Portfolios

Date
2022-01-03    1.048487
2022-01-04    1.032020
2022-01-05    1.006086
2022-01-06    1.002026
2022-01-07    0.996787
                ...   
2024-08-21    2.116223
2024-08-22    2.064162
2024-08-23    2.118041
2024-08-26    2.086697
2024-08-27    2.104627
Length: 666, dtype: float64

In [43]:
import itertools

def calbiration(start_date, end_date, params):
    sumamry = []

    for p in params:
        portfolio = PortfolioStandard(
            capital=169336.48958837058,
            name='MVOPortfolio',        
            rebalance_iter=RebalancerIter('0 0 * * Fri', 2),        
            tc_model=TransactionCostFutu(), 
            systems=[                                        
                TradingSubSystemSingle(vol_target=vol_target, instruments=instruments,  strategy=[MeanVarianceOptLGBM(gamma=10, hhi=0.2, lookback=10, confidence=1, train_days=p[0], forecast=p[1])], max_leverage=max_leverage, offset=60),                
            ]
        )
        portfolio.set_start_date(start_date)
        portfolio.set_end_date(end_date)
        portfolio.backtest_subsystems()

        ret = (portfolio.ret).mean(axis=1)
        res = performance_summary(ret)
        del res['strategy']
        res['train_days'] = p[0]
        res['forecast'] = p[1]
        sumamry.append(res)

    summary = pd.DataFrame(sumamry)
    summary = summary.set_index(['train_days', 'forecast'])    
    return summary

start_date = pd.to_datetime(datetime(2023,1,1))
end_date = get_today()

forcasts = np.arange(1,20,2)
train_days = forcasts * 3
params = list(itertools.product(train_days, forcasts))
params = [[p[0], p[1]] for p in params if p[0] > p[1]]

summary = calbiration(start_date, end_date, params)

2024-08-28 02:41:01,504 - TradingSubSystemSingle [MVO - LGBM1] - INFO - Generating position for strategy MVO - LGBM1 between 2022-12-30 and 2024-08-27......
100%|██████████| 433/433 [00:41<00:00, 10.49it/s]
2024-08-28 02:41:42,847 - TradingSubSystemSingle [MVO - LGBM1] - INFO - Volatility Target = 25.0% | Price Volatility = 34.7% | Last Scale Factor = 0.71
2024-08-28 02:41:42,852 - TradingSubSystemSingle [MVO - LGBM1] - INFO - Generating position for strategy MVO - LGBM1 between 2022-12-30 and 2024-08-27......
100%|██████████| 433/433 [00:44<00:00,  9.78it/s]
2024-08-28 02:42:27,176 - TradingSubSystemSingle [MVO - LGBM1] - INFO - Volatility Target = 25.0% | Price Volatility = 30.1% | Last Scale Factor = 0.83
2024-08-28 02:42:27,183 - TradingSubSystemSingle [MVO - LGBM1] - INFO - Generating position for strategy MVO - LGBM1 between 2022-12-30 and 2024-08-27......
100%|██████████| 433/433 [00:42<00:00, 10.30it/s]
2024-08-28 02:43:09,268 - TradingSubSystemSingle [MVO - LGBM1] - INFO - Vol

In [54]:
summary.groupby(['forecast']).mean(numeric_only=True)

,cumulative_return,annualized_return,annualized_volatility,annualized_sharpe_ratio,maximum_drawdown
forecast,,,,,
1,2.394605,0.554438,0.225124,2.313560,-0.165965
3,2.573938,0.599150,0.229350,2.467409,-0.168775
5,2.425563,0.564480,0.230724,2.303390,-0.161695
7,2.398443,0.557701,0.231005,2.271699,-0.169183
9,2.298882,0.531972,0.231108,2.157372,-0.173617
11,2.308036,0.534624,0.231370,2.167157,-0.179597
13,2.433484,0.566173,0.233472,2.284050,-0.174259
15,2.362052,0.548086,0.235180,2.189240,-0.176754
17,2.433564,0.564780,0.235816,2.255289,-0.175898


In [51]:
summary.groupby(['train_days']).mean(numeric_only=True)

,cumulative_return,annualized_return,annualized_volatility,annualized_sharpe_ratio,maximum_drawdown
train_days,,,,,
3,2.009155,0.445718,0.208479,1.977374,-0.160107
9,2.574131,0.598871,0.220841,2.560241,-0.137235
15,2.387184,0.554167,0.225952,2.304723,-0.189160
21,2.373117,0.551453,0.229945,2.252544,-0.187102
27,2.413475,0.562298,0.234285,2.258118,-0.172186
33,2.335028,0.542349,0.236969,2.148111,-0.180386
39,2.248509,0.520684,0.238451,2.043373,-0.206229
45,2.413377,0.560853,0.230327,2.291514,-0.160017
51,2.501153,0.581578,0.231390,2.368046,-0.159194
